# Advanced Tutorial on metasyn

In this tutorial, we will be creating synthetic data using the `metasyn` package.

Some advanced features of metasyn will be covered further along the tutorial, such as handling dates, setting distributions, ensuring uniqueness in columns and adding variable descriptions.

For more information refer to the [user's guide](https://metasynth.readthedocs.io/en/latest/usage/usage.html) on the docs.

## Step 0: Install the metasyn package and import required packages

First, let's install the metasyn package.

In [1]:
# Run the following line to install metasyn
# %pip install metasyn

Now, let's import the required packages.

In [2]:
# import required packages
from pathlib import Path

import polars as pl

import metasyn as ms
from metasyn import MetaFrame, VarSpec, demo_file

## Step 1: Loading the dataset

The first step to create synthetic data is to load your dataset into a DataFrame. For this tutorial, we will be using the [Titanic dataset](https://www.kaggle.com/c/titanic/data), which can easily be accessed through the metasyn `demo_file()` function. 

It is important to set the data types of columns in the DataFrame correctly, as this will help metasyn to infer the correct distributions for each variable later.


> **Note** 
> In this tutorial we use [Polars](https://pola.rs) to create the DataFrame, as that is what metasyn uses internally. Pandas is also supported, but will automatically be converted to Polars by metasyn. For best results it is recommended to use Polars.

In [3]:
# get the demonstration data file
csv_path = demo_file("titanic")

# ensure columns are of the correct type
data_types = {"Sex": pl.Categorical, "Embarked": pl.Categorical}

# read the data from the csv path
df, file_format = ms.read_csv(csv_path, schema_overrides=data_types)

# check out the data
df.head()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Owen Harris""","""male""",22,0,"""A/5 21171""",7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. John Bradley (Fl…","""female""",38,0,"""PC 17599""",71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Miss. Laina""","""female""",26,0,"""STON/O2. 3101282""",7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs. Jacques Heath (…","""female""",35,0,"""113803""",53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. William Henry""","""male""",35,0,"""373450""",8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


We can check the data types of our DataFrame as follows:

In [4]:
df.schema

Schema([('PassengerId', Int64),
        ('Name', String),
        ('Sex', Categorical(ordering='physical')),
        ('Age', Int64),
        ('Parch', Int64),
        ('Ticket', String),
        ('Fare', Float64),
        ('Cabin', String),
        ('Embarked', Categorical(ordering='physical')),
        ('Birthday', Date),
        ('Board time', Time),
        ('Married since', Datetime(time_unit='us', time_zone=None)),
        ('all_NA', String)])

We see that most variables are now nicely specified as strings, categories, dates and ints where necessary. We can get some more information on the DataFrame by calling the `describe()` on it, this will give us some information on the distribution of the variables:  

In [5]:
df.describe()

statistic,PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
str,f64,str,str,f64,f64,str,f64,str,str,str,str,str,str
"""count""",891.0,"""891""","""891""",714.0,891.0,"""891""",891.0,"""204""","""889""","""813""","""812""","""799""","""0"""
"""null_count""",0.0,"""0""","""0""",177.0,0.0,"""0""",0.0,"""687""","""2""","""78""","""79""","""92""","""891"""
"""mean""",446.0,null,null,29.693277,0.381594,null,32.204208,null,null,"""1921-07-27 22:08:24.798000""","""14:38:10.014778""","""2022-07-31 03:43:48.767209""",null
"""std""",257.353842,null,null,14.524527,0.806057,null,49.693429,null,null,null,null,null,null
"""min""",1.0,"""Abbing, Mr. Anthony""",null,0.0,0.0,"""110152""",0.0,"""A10""",null,"""1903-07-28""","""10:39:40""","""2022-07-15 12:21:15""",null
"""25%""",224.0,null,null,20.0,0.0,null,7.925,null,null,"""1911-09-18""","""12:39:02""","""2022-07-23 11:16:56""",null
"""50%""",446.0,null,null,28.0,0.0,null,14.4542,null,null,"""1922-03-26""","""14:29:34""","""2022-07-31 00:36:56""",null
"""75%""",669.0,null,null,38.0,0.0,null,31.0,null,null,"""1930-08-29""","""16:40:12""","""2022-08-08 03:35:52""",null
"""max""",891.0,"""van Melkebeke, Mr. Philemon""",null,80.0,6.0,"""WE/P 5735""",512.3292,"""T""",null,"""1940-05-27""","""18:39:28""","""2022-08-15 10:32:05""",null


## Step 2: Generating a MetaFrame

Now that we have properly formatted our DataFrame, we can easily generate a MetaFrame for it. 
We'll do this without passing in any optional parameters, but later on in this tutorial we will cover how custom parameters can help provide control over the MetaFrame generation process. 

> **MetaFrames:**
> A MetaFrame is an object which captures the essential aspects of the dataset, including variable names, types, data types, the percentage of missing values, and distribution parameters. MetaFrame objects capture all the information needed to generate a synthetic dataset that aligns with the original dataset, without containing any *entries* of the original dataset.

More information on generating MetaFrames can be found on the metasyn docs ['generating metaframes'](https://metasynth.readthedocs.io/en/latest/usage/generating_metaframes.html) page.

Generating a MetaFrame is simple, and can be done by simply calling the `MetaFrame.fit_dataframe()` class method, passing in the DataFrame as a parameter.

In [6]:
# Generate and fit a MetaFrame to the DataFrame
mf = MetaFrame.fit_dataframe(df, file_format=file_format)

100%|██████████| 13/13 [00:04<00:00,  3.21it/s]


We can use the built-in Python `print` function to display the (statistical) metadata contained in the MetaFrame in an easy-to-read format:

In [7]:
print(mf)

# Rows: 891
# Columns: 13

Column 1: "PassengerId"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.unique_key
	- Provenance: builtin
	- Parameters:
		- lower: 1
		- consecutive: True
	

Column 2: "Name"
- Variable Type: string
- Data Type: String
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.freetext
	- Provenance: builtin
	- Parameters:
		- locale: EN
		- avg_sentences: 2.4691358024691357
		- avg_words: 4.093153759820426
	

Column 3: "Sex"
- Variable Type: categorical
- Data Type: Categorical(ordering='physical')
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.multinoulli
	- Provenance: builtin
	- Parameters:
		- labels: ['female' 'male']
		- probs: [0.35241302 0.64758698]
	

Column 4: "Age"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.1987
- Distribution:
	- Type: core.truncated_normal
	- Provenance: builtin
	- Parameters:
		- lower: -1e-08
	

## Step 3: Exporting the MetaFrame

After creating the MetaFrame, Metasyn can serialize and save it into a GMF file using `mf.save()`, passing in the filepath as a parameter. 


> **GMF files:**
> GMF files are JSON files that follow the [Generative Metadata Format (GMF)](https://github.com/sodascience/generative_metadata_format), a format designed to contain statistical metadata for (tabular) datasets that has been designed to be easy to read and understand. This allows users to audit, understand, modify and share their data generation model with ease.

More information on saving and importing MetaFrames can be found on the metasyn docs ['saving and importing metaframes'](https://metasynth.readthedocs.io/en/latest/usage/saving_metaframes.html) page.

In [8]:
file_path = Path("gmf_files", "example_gmf_titanic.json")

# Serialize and save the MetaFrame to a GMF file
mf.save(file_path)

The GMF file should now be saved to the specified filepath, feel free to open and inspect it!

It's also possible to preview how the saved file would look, without actually saving it to disk. This can be done as follows:

In [9]:
# Get a preview of the GMF file (`repr()`) and print it (`print()`)
print(repr(mf))

MetaFrame: size = (891 x 13) <MetaVar <PassengerId, core.unique_key>, MetaVar <Name, core.freetext>, ...>


A GMF file can be imported and loaded into a MetaFrame using the `MetaFrame.load()` class method, passing in the file path as a parameter. 

In [10]:
# Create a MetaFrame based on a GMF (.json or .toml) file
mf = MetaFrame.load(file_path)

## Step 4: Generating synthetic data

Once a MetaFrame is loaded, synthetic data can be generated from it. We can do so by using the the `synthesize` method of the MetaFrame, passing in how many rows the generated data should contain as a parameter. This returns a DataFrame with the synthetic data.

More information on generating synthetic data based on MetaFrames can be found on the metasyn docs, [here](https://metasynth.readthedocs.io/en/latest/usage/generating_synthetic_data.html).

In [11]:
# generate synthetic data
syn_df = mf.synthesize(5)

We can now view the synthetic data:

In [12]:
syn_df

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Space. No.""","""female""",null,0,"""4675""",41.424338,null,"""S""",1922-09-27,11:57:22,2022-08-13 22:21:36,null
2,"""Soldier. Likely.""","""female""",43,0,"""525059""",60.518986,null,"""S""",1913-08-27,14:17:03,null,null
3,"""College. Black bad.""","""male""",15,0,"""15118""",18.625399,"""G14""","""S""",1933-09-07,16:01:34,2022-08-09 11:11:03,null
4,"""Heart. Manager.""","""male""",null,0,"""0759""",104.760631,null,"""S""",1914-01-17,11:58:42,2022-07-24 18:00:05,null
5,"""Produce answer.""","""female""",51,0,"""655565""",15.550622,"""E116""","""S""",1938-05-11,13:54:44,2022-07-19 13:15:33,null


As you can see, the synthetic data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

The `MetaFrame.fit_dataframe()` method allows you to have more control over how your synthetic dataset is generated by passing in an optional `spec` (short for specification) parameter. `spec` is a dictionary that can be used to give metasyn instructions on a per-variable basis, these instructions can range from setting a variable to be unique, to directly setting its distribution. 

### Spec: Fake names (and other Faker data types)

Currently, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in metasyn is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, metasyn supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We can specify metasyn to use Faker names for the `Name` column as follows:

In [13]:
# First, we create a specification dictionary for the variables
from metasyn.distribution import FakerDistribution

var_specs = [
    VarSpec("Name", distribution=FakerDistribution("name")),
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs)
mf.synthesize(5)

100%|██████████| 13/13 [00:01<00:00,  8.11it/s]


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Scott Vance""","""female""",28,0,"""285336""",7.135904,null,"""C""",1917-11-12,14:24:41,2022-08-12 13:29:01,null
2,"""Melissa Harris""","""female""",19,0,"""711977""",7.623058,null,"""S""",1913-12-05,11:03:18,2022-07-31 14:36:23,null
3,"""Monica Moody""","""male""",null,0,"""20889""",6.163413,null,"""S""",1912-01-22,12:19:35,2022-07-26 01:52:18,null
4,"""Kyle Hopkins""","""female""",null,0,"""912032""",0.673363,null,"""Q""",1916-06-14,17:07:32,2022-08-09 13:51:51,null
5,"""Amber Gibson""","""male""",24,0,"""55788""",8.299929,null,"""S""",1923-07-05,13:47:43,2022-07-22 22:40:49,null


That already looks a lot better for the `Name` column!

### Spec: Setting distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the distribution with the best fit from all available distributions for the variable type. However, we can also manually specify which distribution to fit, or simply specify the distribution including the parameters for the variable.

In [14]:
from metasyn.distribution import DiscreteUniformDistribution

var_specs = [
    VarSpec("Name", distribution=FakerDistribution("name")),
    VarSpec("Fare", distribution="lognormal"), # estimate / fit an exponential distribution based on the data
    VarSpec("Age", distribution=DiscreteUniformDistribution(20, 40)) # fully specify a distribution for age (uniform between 20 and 40)
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs)
mf.synthesize(5)

100%|██████████| 13/13 [00:01<00:00, 10.40it/s]


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Christopher Davis""","""female""",32,0,"""6838""",7.876914,null,"""S""",1916-02-22,13:15:22,2022-08-04 01:02:32,null
2,"""Randy Smith""","""female""",35,0,"""797807""",3.752958,null,"""S""",1937-02-02,17:00:44,2022-08-01 22:26:42,null
3,"""Nicole Ryan""","""male""",null,0,"""21081""",1.593362,null,"""S""",1921-10-07,16:30:39,2022-07-19 11:29:42,null
4,"""Willie Ruiz""","""female""",38,0,"""015258""",0.811072,"""F639""","""S""",1931-08-07,18:31:18,2022-07-18 19:36:41,null
5,"""Dennis Ballard""","""male""",37,0,"""02785""",0.292984,null,"""S""",1915-10-11,16:10:54,null,null


### Spec: Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [26]:
from metasyn.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[A-F][0-9]{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [A-F]?(\d{2,3})?


var_specs = [
    VarSpec("Name", distribution=FakerDistribution("name")),
    VarSpec("Fare", distribution="lognormal"), # estimate / fit an exponential distribution based on the data
    VarSpec("Age", distribution=DiscreteUniformDistribution(20, 40)), # fully specify a distribution for age (uniform between 20 and 40)
    VarSpec("Cabin", distribution=cabin_distribution),  # Use our previously defined distribution
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs, file_format=file_format)
mf.synthesize(10)

100%|██████████| 13/13 [00:01<00:00,  8.90it/s]


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""John Cooper""","""male""",32,0,"""31445""",0.170095,null,"""S""",1911-10-01,12:11:37,2022-07-31 18:05:58,null
2,"""Kristin Sanchez""","""female""",30,1,"""8441""",1.783395,null,"""Q""",1927-04-30,17:48:31,2022-08-07 17:20:28,null
3,"""Laura Cardenas""","""male""",26,0,"""8004""",0.796495,null,"""C""",1935-05-25,16:21:57,2022-07-16 11:01:03,null
4,"""Michael Dunn""","""male""",37,0,"""0418""",1.171866,null,"""S""",1904-10-21,13:45:39,2022-07-22 02:29:53,null
5,"""Bradley Page""","""female""",31,0,"""753948""",1.546473,null,"""S""",1908-04-07,17:46:26,2022-08-05 19:35:17,null
6,"""Cody Gonzales""","""male""",22,0,"""320200""",1.095945,null,"""C""",1903-11-28,14:36:49,2022-08-14 00:55:01,null
7,"""Katherine Perez""","""male""",null,0,"""04394""",1.599978,null,"""Q""",1927-03-09,16:50:44,2022-08-06 10:03:30,null
8,"""William Webster""","""female""",null,0,"""NC 7458""",2.73275,"""D82""","""Q""",1923-10-08,12:32:57,2022-07-17 15:29:42,null
9,"""Micheal Bennett""","""male""",23,0,"""085287""",1.248871,null,"""C""",1912-05-27,14:29:31,2022-08-08 12:02:45,null


## Step 6: Comparing the final synthetic dataset to the original

Let's first compare the averages of the numerical columns:

In [16]:
df.mean()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,str,f64,str,cat,datetime[ms],time,datetime[μs],str
446.0,null,null,29.693277,0.381594,null,32.204208,null,null,1921-07-27 22:08:24.798,14:38:10.014778325,2022-07-31 03:43:48.767209,null


In [17]:
mf.synthesize().mean()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,str,f64,str,cat,datetime[ms],time,datetime[μs],str
446.0,null,null,29.588905,0.072952,null,1.621857,null,null,1921-07-25 22:38:01.189,14:47:42.330413016,2022-07-30 13:56:19.170854,null


Then, we can also see how many missing values are in each column

In [18]:
df.null_count()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,177,0,0,0,687,2,78,79,92,891


In [19]:
mf.synthesize().null_count()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,177,0,0,0,680,1,72,70,117,891


## Step 7: Adding descriptions to variables

With the data being taken care of, we can still do one last thing. We can add descriptions to the variables, to clarify what they mean. This can be particularly useful when sharing the `MetaFrame` or generated data with others, as it gives them more context to what they're working with.

One way of adding a description to a variable, is by setting it in the `spec` dictionary, this can be done by simply adding a `description` key with the description as a value. For example, adding a description to the `Cabin` column can be done as follows:

In [20]:
var_specs = [
    # Utilize the Faker library to synthesize realistic names for the `Name` column
    VarSpec("Name", distribution=FakerDistribution("name")),

    # Fit `Fare` to an log-normal distribution, but base the parameters on the data
    VarSpec("Fare", distribution="lognormal"), # estimate / fit an exponential distribution based on the data

    # Set the `Age` column to a discrete uniform distribution ranging from 20 to 40
    VarSpec("Age", distribution=DiscreteUniformDistribution(20, 40)), # fully specify a distribution for age (uniform between 20 and 40)

    # Use a regex-based distribution to generate `Cabin` values following [A-F][0-9]{2,3}
    VarSpec("Cabin", distribution=cabin_distribution, description="The cabin number of the passenger."),  # Use our previously defined distribution
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs)

100%|██████████| 13/13 [00:01<00:00, 12.88it/s]


We can get a list of all the descriptions in the fitted `MetaFrame` by accessing its `descriptions` property, as follows:

In [21]:
print(mf.descriptions)

{'Cabin': 'The cabin number of the passenger.'}


Instead of setting the description in the variable specification (which happens before fitting a `MetaFrame` to a `DataFrame`), we can assign a description to an already generated `MetaFrame` by directly setting a column's description attribute. For example, we can assign a description to the `PassengerId` column as follows:

In [22]:
mf["PassengerId"].description = "The ID of each passenger, as assigned by Pandas."

print(mf.descriptions)

{'PassengerId': 'The ID of each passenger, as assigned by Pandas.', 'Cabin': 'The cabin number of the passenger.'}


We can also set multiple descriptions of an already generated `MetaFrame` at once by passing in a dictionary of descriptions to its `descriptions` property. For example, we can set descriptions for the `Age` and `Name` columns as follows:

In [23]:
mf.descriptions = {"Name": "Name of the passenger", "Age": "Age of the passenger in years"}

print(mf.descriptions)

{'PassengerId': 'The ID of each passenger, as assigned by Pandas.', 'Name': 'Name of the passenger', 'Age': 'Age of the passenger in years', 'Cabin': 'The cabin number of the passenger.'}


Instead of a dictionary, it is also possible to pass in a list of descriptions to the `descriptions` property of a `MetaFrame`. 

This can only be done if the list has the same length as the number of variables. In other words, each description must be passed in. 

This can be useful for example when generating placeholder descriptions automatically through list comprehension, as is done in the following example:

In [24]:
mf.descriptions = [f"Placeholder description for {var.name}" for var in mf.meta_vars]

print(mf.descriptions)

{'PassengerId': 'Placeholder description for PassengerId', 'Name': 'Placeholder description for Name', 'Sex': 'Placeholder description for Sex', 'Age': 'Placeholder description for Age', 'Parch': 'Placeholder description for Parch', 'Ticket': 'Placeholder description for Ticket', 'Fare': 'Placeholder description for Fare', 'Cabin': 'Placeholder description for Cabin', 'Embarked': 'Placeholder description for Embarked', 'Birthday': 'Placeholder description for Birthday', 'Board time': 'Placeholder description for Board time', 'Married since': 'Placeholder description for Married since', 'all_NA': 'Placeholder description for all_NA'}


## The end

That's it for this tutorial! You should now have a good understanding of how to use metasyn to generate synthetic data from a dataset. If you want to learn more, check out the [metasyn docs](https://metasynth.readthedocs.io/en/latest/).

If you have any questions, feel free to [reach out](https://metasynth.readthedocs.io/en/latest/about/contact.html).

